In [ ]:
import sys
import os

sys.path.append('../')
from dataloader.MiddleburyLoader import myImageFloder
from notebooks.notebook_utils import visualize_sample, download_sample, list_s3

In [ ]:
train_list = list_s3('airsim_dataset/')['CommonPrefixes']

In [ ]:
def walk_through_dataset(dataset_list):
    for example in dataset_list:
        example = example['Prefix']
        example = 's3://autogpe-datasets/{}'.format(example)
        folders = example.split('/')[-2:]
        folder = '/'.join(folders)
        out_folder = '../data/airsim/{}'.format(folder)
        print('sync: {}'.format(example))
        download_sample(example, out_folder)
        left_train = []
        right_train = []
        disp_train_l = []
        _, orientations, _ = list(os.walk(out_folder))[0]
        for orientation in orientations:
            left_train.append(os.path.join(out_folder, *[orientation, 'left_bgr.png']))
            right_train.append(os.path.join(out_folder, *[orientation, 'right_bgr.png']))
            disp_train_l.append(os.path.join(out_folder, *[orientation, 'left_disparity.npy']))
        rand_scale = [0.5, 1.2]
        airsim_dataloader = myImageFloder(left_train, right_train, disp_train_l, rand_scale=rand_scale, order=0)   
        for (img_l, img_r, disp_l) in airsim_dataloader:
            visualize_sample(img_l, img_r, disp_l)
            yield

In [ ]:
train_iter = walk_through_dataset(train_list)

In [ ]:
next(train_iter)